In [1]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles

In [2]:
joint_list2 = [[8, 7, 6, 5], [12, 11, 10, 9], [16, 15, 14, 13], [20, 19, 18, 17]]

In [3]:
def decide_gesture(inout_list):
    
    if all([not(x ^ y)for x, y in zip(inout_list,[True, False, True, True, True])]):
        return 'one'
    elif all([not(x ^ y)for x, y in zip(inout_list,[True, False, False, True, True])]):
        return 'two'
    elif all([not(x ^ y)for x, y in zip(inout_list,[True, False, False, False, True])]):
        return 'three'
    elif all([not(x ^ y)for x, y in zip(inout_list,[True, False, False, False, False])]):
        return 'four'
    elif all([not(x ^ y)for x, y in zip(inout_list,[False, False, False, False, False])]):
        return 'five'
    elif all([not(x ^ y)for x, y in zip(inout_list,[False, True, True, True, False])]):
        return 'six'
    elif all([not(x ^ y)for x, y in zip(inout_list,[False, False, True, True, True])]):
        return 'seven'
    elif all([not(x ^ y)for x, y in zip(inout_list,[False, False, False, True, True])]):
        return 'eight'
    elif all([not(x ^ y)for x, y in zip(inout_list,[True, False, True, True, False])]):
        return 'rock1'
    elif all([not(x ^ y)for x, y in zip(inout_list,[False, False, True, True, False])]):
        return 'rock2'

In [4]:
def formula_big(x1, x2, x3, y1, y2, y3):
    
    mul_x = 640
    mul_y = 480
    x1 = x1 * mul_x
    x2 = x2 * mul_x
    x3 = x3 * mul_x
    y1 = y1 * mul_y
    y2 = y2 * mul_y
    y3 = y3 * mul_y
    
    a = (y1 - y2) / (x1 - x2)
    b = y1 - a * x1
    
    #cv2.line(image, (int(round((100 - b) / a)), 100), (int(round((480 - b) / a)), 640), (0, 0, 255), 1)
    
    if x3 * a + b > y3:
        if x1 > x2:
            return True
        else:
            return False
    if x1 > x2:
        return False
    else:
        return True

In [5]:
def formula_other(x1, x2, x3, y1, y2, y3):
    
    mul_x = 640
    mul_y = 480
    x1 = x1 * mul_x
    x2 = x2 * mul_x
    x3 = x3 * mul_x
    y1 = y1 * mul_y
    y2 = y2 * mul_y
    y3 = y3 * mul_y
    
    a = (y1 - y2) / (x1 - x2)
    b = y1 - a * x1
    
    #cv2.line(image, (int(round((100 - b) / a)), 100), (int(round((480 - b) / a)), 640), (0, 0, 255), 1)
    
    
    if x1 > x2:
        if x3 * a + b > y3:
            return True
        return False
    else:
        if x3 * a + b > y3 + 30:
            return False
        return True
            

In [6]:
import math

def draw_and_get_finger_angles2(image, results, joint_list):
    
    pos = 20
    RLmultiply = 0
    distance = None
    gesture = None
    inout = ''
    # Loop through hands
    for RL, hand in enumerate(results.multi_hand_landmarks):
        inout_list = []
        try:
            if results.multi_handedness[RL].classification[0].label == 'Right':
                RLmultiply = 1
            else:
                RLmultiply = 0
        except:
            pass
        a = np.array([hand.landmark[4].x, hand.landmark[4].y]) #大拇指尖
        b = np.array([hand.landmark[8].x, hand.landmark[8].y]) #食指尖
        c = np.array([hand.landmark[12].x, hand.landmark[12].y]) #中指尖
        d = np.array([hand.landmark[16].x, hand.landmark[16].y]) #無名指尖
        e = np.array([hand.landmark[20].x, hand.landmark[20].y]) #小指尖
        f = np.array([hand.landmark[5].x, hand.landmark[5].y]) #食指根
        g = np.array([hand.landmark[9].x, hand.landmark[9].y]) #中指根
        h = np.array([hand.landmark[13].x, hand.landmark[13].y]) #無名指根
        i = np.array([hand.landmark[17].x, hand.landmark[17].y]) #小指根
        j = np.array([hand.landmark[0].x, hand.landmark[0].y]) #手腕
        
        #大拇指
        inout_list.append(formula_big(f[0], j[0], a[0], f[1], j[1], a[1]))
        #食指
        inout_list.append(not formula_other(f[0], g[0], b[0], f[1], g[1], b[1]))
        #中指
        inout_list.append(not formula_other(g[0], h[0], c[0], g[1], h[1], c[1]))
        #無名指
        inout_list.append(not formula_other(h[0], i[0], d[0], h[1], i[1], d[1]))
        #小拇指
        inout_list.append(not formula_other(h[0], i[0], e[0], h[1], i[1], e[1]))
        
        
        cv2.putText(image, " ".join(str(x) for x in inout_list), (pos + num * 80 + RLmultiply * 320, 20),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        if RLmultiply == 1:
            distance = math.sqrt((hand.landmark[4].x - hand.landmark[12].x)**2 + (hand.landmark[4].y - hand.landmark[12].y)**2)
            cv2.putText(image, str(round(distance, 2)), (int(hand.landmark[0].x * 640), int(hand.landmark[0].y * 480) + 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, inout, (int(hand.landmark[0].x * 640), int(hand.landmark[0].y * 480) + 60),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            
        if RLmultiply == 0:
            gesture = decide_gesture(inout_list)
            cv2.putText(image, gesture, (int(hand.landmark[0].x * 640), int(hand.landmark[0].y * 480) + 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, inout, (int(hand.landmark[0].x * 640), int(hand.landmark[0].y * 480) + 60),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
    return image, distance, gesture

In [7]:
def draw_line(image,left_WRIST_y_coord):
    start_point = (0, left_WRIST_y_coord)
    end_point = (640, left_WRIST_y_coord)
    color = (0, 255, 0)
    thickness = 2
    image = cv2.line(image, start_point, end_point, color, thickness)
    return image

In [8]:
chord_mode_0 = {
    'one':"C",
    'two':"A",
    'three':"D",
    'four':"G"
}
chord_mode_1 = {
    'one':"1",
    'two':"2",
    'three':"3",
    'four':"4"
}
chord_modes = [chord_mode_0, chord_mode_1]

In [9]:
def change_chord_mode(chord_mode):
    num_of_mode = len(chord_modes)-1
    if chord_mode+1 <= num_of_mode:
        return chord_mode + 1
    else:
        return 0

In [10]:
def find_WRIST_coord(image, results):
    left_WRIST_y_coord = None
    right_WRIST_y_coord = None
    for RL, hand in enumerate(results.multi_hand_landmarks):
        if results.multi_handedness[RL].classification[0].label == 'Left':
            
            left_WRIST_y_coord = int(hand.landmark[mp_hands.HandLandmark.WRIST].y * 480)
            image = draw_line(image,left_WRIST_y_coord)
            
        else:
            right_WRIST_y_coord = int(hand.landmark[mp_hands.HandLandmark.WRIST].y * 480)
        
    return image, left_WRIST_y_coord, right_WRIST_y_coord

In [11]:
from playsound import playsound
import threading
import time

def init_play_chord(chord):
    print("Thread", chord)
    playsound('music/{}_chord.mp3'.format(chord)) # 讓初次載入時發出聲音
    
def play_chord(chord):
    threads[chords[chord]] = threading.Thread(target = init_play_chord, args = (chord,))
    threads[chords[chord]].start()

chords = {
    "C": 0,
    "A": 1,
    "D": 2,
    "G": 3,
    "1": 0,
    "2": 1,
    "3": 2,
    "4": 3
}

# 建立 4 個子執行緒
threads = []
for i, chord_name in enumerate(chords):
    threads.append(threading.Thread(target = init_play_chord, args = (chord_name,)))
    threads[i].start()
    time.sleep(2)
    
for i in range(4):
    threads[i].join()

Thread C
Thread A
Thread D
Thread G
Thread 1
Thread 2
Thread 3
Thread 4


In [13]:
cap = cv2.VideoCapture(0)
counter = 0 # 計算偵數，讓模式間隔 N 偵以上才能切換
chord_mode = 0
N = 50
threshold = 0.2 
Status = True

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()
        
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )
                image, left_WRIST_y_coord, right_WRIST_y_coord = find_WRIST_coord(image, results)
                #print("left_WRIST_y_coord: ",left_WRIST_y_coord,"right_WRIST_y_coord: ",right_WRIST_y_coord,)
        
            # Draw angles to image from joint list
            image, distance, gesture = draw_and_get_finger_angles2(image, results, joint_list2)
            
        # Save our image    
        #cv2.imwrite(os.path.join('Output Images', '{}.jpg'.format(uuid.uuid1())), image)
        
        try:
            if right_WRIST_y_coord < left_WRIST_y_coord:
                Status = True          

            if gesture and right_WRIST_y_coord > left_WRIST_y_coord and Status: 
                play_chord(chord_modes[chord_mode][gesture])
                counter = 0
                Status = False

#             if distance > threshold:
#                 Status = True
#             if gesture and distance < threshold and counter > N and Status: # counter>20 讓音樂間隔 N 偵以上播放
#                 play_chord(num2chord[gesture])
#                 counter = 0
#                 Status = False
        except:
            try:
                if gesture == 'rock1' and counter > N : # counter>20 讓模式間隔 N 偵以上才能切換
                    chord_mode = change_chord_mode(chord_mode)
                    print("change chord mode!")
                    counter = 0
                elif gesture == 'rock1':
                    text = 'CHMOD'
                    cv2.putText(image, text, (150, 200), cv2.FONT_HERSHEY_SIMPLEX,
                      3, (0, 255, 255), 9, cv2.LINE_AA)
            except:
                pass
            
        cv2.putText(image, 'chord mode: {}'.format(chord_mode), (380, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)        
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
        counter = counter +1 

cap.release() 
cv2.destroyAllWindows()

Thread G
change chord mode!
Thread 4
Thread 4
Thread 4
Thread 1
change chord mode!
change chord mode!
change chord mode!
Thread C
change chord mode!
Thread 1
change chord mode!
change chord mode!
Thread 1
change chord mode!
Thread C
Thread C
Thread C
Thread C
Thread C
Thread C
Thread C
Thread A
Thread A
Thread D
Thread D
Thread G
Thread G
Thread G
Thread G
Thread G
Thread G
Thread G
Thread G
Thread G
Thread D
Thread G
Thread G
Thread A
Thread D
Thread D
Thread A



    Error 263 for command:
        close music/A_chord.mp3
    指定的裝置未開啟，或無法由 MCI 所辨認。
Failed to close the file: music/A_chord.mp3


Thread C
Thread D
Thread G
Thread G
Thread D
Thread C
Thread D
Thread G



    Error 263 for command:
        close music/D_chord.mp3
    指定的裝置未開啟，或無法由 MCI 所辨認。
Failed to close the file: music/D_chord.mp3


change chord mode!
change chord mode!
change chord mode!
change chord mode!
Thread C
change chord mode!
Thread 1
Thread 1
Thread 1
Thread 1
Thread 2
Thread 1
Thread 2
Thread 3
Thread 4
Thread 4
